In [1]:
pip install yfinance

     ---------------------------------------- 59.7/59.7 kB 1.6 MB/s eta 0:00:00
     -------------------------------------- 112.2/112.2 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Without using Market Indicators


In [1]:
import yfinance as yf
import pandas as pd

# List of stock tickers to trade
stocks = ['BHARTIARTL.NS', 'SBIN.NS', 'DRREDDY.NS', 'GSPL.NS', 'JSWSTEEL.NS', 'JSWENERGY.BO',
          'RELIANCE.NS', 'YESBANK.NS', 'TATAMOTORS.NS', 'LAURUSLABS.NS', 'NTPC.NS', 'SAIL.BO']

# the initial wallet amount
wallet = 100000


transactions = []

for stock in stocks:
    data = yf.download(stock, start="2015-01-01", end="2022-12-31")
    data['Prev_Close'] = data['Close'].shift(1)
    data = data.dropna()
    
    units = 0
    for date, row in data.iterrows():
        balance = wallet - (units * row['Open'])
        if balance >= row['Open'] and row['Open'] < row['Prev_Close']:
            # Buy stocks
            num_units = int(balance / row['Open'])
            units += num_units
            wallet -= num_units * row['Open']
            transactions.append([date.strftime('%Y-%m-%d'), stock, round(row['Open'], 2), 'BUY', num_units])
            
        elif units > 0 and (row['Open'] > row['Prev_Close'] or row['Close'] > row['Prev_Close']):
            # Sell stocks
            num_units = units
            units = 0
            wallet += num_units * row['Close'] * 0.95  # 5% transaction fee on selling
            transactions.append([date.strftime('%Y-%m-%d'), stock, round(row['Close'], 2), 'SELL', num_units])
            
# Calculate the total profit earned by summing up the profits obtained from each transaction
total_profit = sum([((transaction[2] * transaction[4]) * 0.95) if transaction[3] == 'SELL' else 0 for transaction in transactions])

df = pd.DataFrame(transactions, columns=['Date', 'Stock', 'Transaction Price', 'Event', 'Total Units'])

df.to_excel('stock_trading.xlsx', index=False)

print(f'Total Profit: INR {round(total_profit, 2)}')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Total Profit: INR 1919626.66


# Using Market Indicators

In [7]:
import yfinance as yf
import pandas as pd

# List of stock tickers to trade
stocks = ['BHARTIARTL.NS', 'SBIN.NS', 'DRREDDY.NS', 'GSPL.NS', 'JSWSTEEL.NS', 'JSWENERGY.BO',
          'RELIANCE.NS', 'YESBANK.NS', 'TATAMOTORS.NS', 'LAURUSLABS.NS', 'NTPC.NS', 'SAIL.BO']

# Set the initial wallet amount
wallet = 100000

# Initialize an empty list to store the transactions made during the trading journey
transactions = []

# Loop through the historical data of each stock and perform the trading strategy for each day
for stock in stocks:
    data = yf.download(stock, start="2015-01-01", end="2022-12-31")
    data['Prev_Close'] = data['Close'].shift(1)
    data = data.dropna()
    
    units = 0
    for date, row in data.iterrows():
        balance = wallet - (units * row['Open'])
        
        #  the 50-day moving average
        if len(data.loc[:date]) >= 50:
            ma = data.loc[:date]['Close'][-50:].mean()
        else:
            ma = 0
        
        if balance >= row['Open'] and row['Open'] < row['Prev_Close'] and row['Open'] > ma:
            # Buy stocks
            num_units = int(balance / row['Open'])
            units += num_units
            wallet -= num_units * row['Open']
            transactions.append([date.strftime('%Y-%m-%d'), stock, round(row['Open'], 2), 'BUY', num_units])
            
        elif units > 0 and ((row['Open'] > row['Prev_Close'] and row['Open'] > ma) or row['Close'] > ma or row['Open'] < (ma * 0.9)):
            # Sell stocks
            num_units = units
            units = 0
            wallet += num_units * row['Close'] * 0.95  # 5% transaction fee on selling
            transactions.append([date.strftime('%Y-%m-%d'), stock, round(row['Close'], 2), 'SELL', num_units])
            
        elif units > 0 and row['Open'] < (ma * 0.9):
            # Sell stocks due to stop-loss condition
            num_units = units
            units = 0
            wallet += num_units * row['Open'] * 0.95  # 5% transaction fee on selling
            transactions.append([date.strftime('%Y-%m-%d'), stock, round(row['Open'], 2), 'SELL (STOP-LOSS)', num_units])

# Calculate the total profit earned by summing up the profits obtained from each transaction
total_profit = sum([((transaction[2] * transaction[4]) * 0.95) if transaction[3] == 'SELL' or transaction[3] == 'SELL (STOP-LOSS)' else 0 for transaction in transactions])


df = pd.DataFrame(transactions, columns=['Date', 'Stock', 'Transaction Price', 'Event', 'Total Units'])


df.to_excel('stock_trading2.xlsx', index=False)

print(f'Total Profit: INR {round(total_profit, 2)}')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Total Profit: INR 1883855.54
